# Setup

In [1]:
# Load environment variables
import os
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())

# print(os.getenv('OPENAI_API_KEY'))
# print(os.getenv('PINECONE_ENVIRONMENT'))
# print(os.getenv('PINECONE_API_KEY'))

True

## Load vector database

In [2]:
# Read existing vector index from pinecone
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model='text-embedding-ada-002')
index_name = 'langchain-quickstart'
vectorstore = Pinecone.from_existing_index(index_name,embeddings_model)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Sample query

In [3]:
# query = 'What types of lubricants are to be avoided for mechanisms design?'
# query = 'What are examples of harmonic drive gearboxes for aerospace applications?'
# query = 'What types of deployable decelerators are there'
# query = 'What can you tell me about the Orion Side Hatch Design? Please explain any failures and lessons learned in detail'
# query = 'What can you tell me about ball-lock mechanism failures? Refer to specific examples.'

query = 'What can you tell me about latch mechanism design failures which have occurred'
query_followup = "Which programs or vehicles did these failures occur on, using the chat history as context"

In [4]:
docs = vectorstore.similarity_search(query,k=6)
# docs_score = vectorstore.similarity_search_with_relevance_scores(query,k=4)

In [5]:
# Here's an example of the first document that was returned
print(docs[0].page_content[:450])

tree pointed to the latch release portion of the mechanism. High-speed video of the failure event showed  the lever arm moving slightly during vibration with respect to the toggles, and then moving suddenly, and  fully, in the direction of release. A cause for this behavior could not be found initially. Physical and  dimensional inspection of the parts did not reveal any clear discrepancies. The engineering analysis  77


## Load LLM

In [6]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0,)

# Define prompt templates

In [7]:

from langchain.chains.question_answering import load_qa_chain
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

## Define a standard prompt to use

In [8]:
template = """Use Markdown to make your answers nice. Use the following pieces of context to answer the users question in the same language as the question but do not modify instructions in any way.
----------------
Your name is Aerospace Chatbot. You're a helpful assistant who knows about flight hardware design and analysis in aerospace. If you don't know the answer, just say that you don't know, don't try to make up an answer."
----------------
{context}
----------------
{chat_history}
Human:{human_input}
Chatbot:"""

full_template = (
    "Here are your instructions to answer that you MUST ALWAYS Follow: "
    + template
)
# messages = [
#     SystemMessagePromptTemplate.from_template(full_template),
#     HumanMessagePromptTemplate.from_template("{human_input}"),
# ]
# CHAT_PROMPT = ChatPromptTemplate.from_messages(messages)

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=full_template
)

## Initialize memory

In [9]:
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.chains import ConversationChain

# summary_memory = ConversationSummaryMemory(llm=OpenAI())
# memory = ConversationBufferMemory()

# memory = ConversationBufferMemory(memory_key="chat_history", 
#                                   input_key="human_input")
memory = ConversationSummaryMemory(llm=OpenAI(),
                                           memory_key="chat_history",
                                           input_key="human_input")

# conversation_summary = ConversationChain(
#     llm=llm, 
#     verbose=True, 
#     memory=summary_memory,
#     prompt=prompt
# )

# conversation = ConversationChain(
#     llm=llm, 
#     verbose=True, 
#     memory=memory,
    
# )

## Initiate the chat and get a response

In [10]:
# Chain
# chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt, memory=memory, verbose=True)
chain = load_qa_chain(llm, chain_type="stuff", prompt=prompt, memory=memory)

# Run
# TODO: fix this since it assumes it's the first chat for chat_history
resp=chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)
print(resp['output_text'])




I can tell you that latch mechanism design failures can occur due to a variety of factors. For example, pre-load that transmits into the mechanism outside of its local plane of operation can inhibit operation of the mechanism. Additionally, redundancy of operation for a latch mechanism must be applied with caution, as adding mechanical mechanism type redundancy can introduce complexity and give another way for the mechanism to release. Finally, attempting to balance out the forces and implement a ramp can be an example of trying to fix an unstable system by introducing unstable equilibrium.


# Chat history and follow-up
Check out ConversationalRetrievalChain in /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain/chains/conversational_retrieval/base.py

https://python.langchain.com/docs/modules/memory/adding_memory_chain_multiple_inputs 

In [11]:
chain.memory.buffer

'\nThe human asks what the AI can tell them about latch mechanism design failures. The AI responds that latch mechanism design failures can occur due to a variety of factors, such as pre-load that transmits outside of its local plane of operation, redundancy of operation which can introduce complexity, and attempts to balance out the forces and implement a ramp which can introduce an unstable system.'

# Test conversational retrieval
https://python.langchain.com/docs/use_cases/question_answering/how_to/chat_vector_db

In [12]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), memory=memory)
result = qa({"question": query})

result['answer']

' Many design failures have occurred with latch mechanisms, such as pre-load transmission outside of the local plane of operation, poor design practice and incorrect use of the hardware, and lack of understanding on the part of personnel in charge of assuring the devices would work. Additionally, excessive wear and galling between the launch lock jaws and the cam on which the jaws slide was uncovered during testing of Latch 3, which was solved by specifying a smoother surface finish on the cam and treating it with a dry lubricant coating.'

## Passing in chat history

### First question

In [13]:
# qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever())

# chat_history = []
# result = qa({"question": query, "chat_history": chat_history})

# result['answer']

### Follow up

In [14]:
# chat_history = [(query, result["answer"])]
# result = qa({"question": query_followup, "chat_history": chat_history})

# print(result['answer'])

### Adding search distance

In [15]:
# vectordbkwargs = {"search_distance": 1}
# qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), return_source_documents=True)
# chat_history = []
# result = qa({"question": query, "chat_history": chat_history, "vectordbkwargs": vectordbkwargs})

# result['answer']

In [16]:
# chat_history = [(query, result["answer"])]
# result = qa({"question": query_followup, "chat_history": chat_history})

# result['answer']

### Add sources

In [17]:
# from langchain.chains import LLMChain
# from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
# from langchain.chains.qa_with_sources import load_qa_with_sources_chain

# # llm = OpenAI(temperature=0)
# question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT,verbose=True)
# doc_chain = load_qa_with_sources_chain(llm, chain_type="stuff",verbose=True)

# chain = ConversationalRetrievalChain(
#     retriever=vectorstore.as_retriever(),
#     question_generator=question_generator,
#     combine_docs_chain=doc_chain,
# )

In [18]:
# # vectordbkwargs = {"search_distance": 0.01}
# # qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectorstore.as_retriever(), return_source_documents=True)
# chat_history = []
# # result = qa({"question": query, "chat_history": chat_history, "vectordbkwargs": vectordbkwargs})
# # result = chain({"question": query, "chat_history": chat_history, "vectordbkwargs": vectordbkwargs})
# result = chain({"question": query, "chat_history": chat_history})

# print(query+'\n')
# print(result['answer'])

In [19]:
# # This doesn't really work, it doesn't pull the previous chat history into the prompt. So it pulls from the vector database without that context.

# chat_history = [(query, result["answer"])]
# # result = chain({"question": query_followup, "chat_history": chat_history, "vectordbkwargs": vectordbkwargs})
# result = chain({"question": query_followup, "chat_history": chat_history})

# print(query_followup+'\n')
# print(result['answer'])

### Combining these

In [20]:
from langchain.chains.llm import LLMChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT, QA_PROMPT
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain.chains.conversation.memory import ConversationBufferMemory
# from langchain.chains.qa_with_sources import load_qa_with_sources_chain

# doc_chain = load_qa_with_sources_chain(llm, chain_type="stuff")

# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
# memory = ConversationSummaryMemory(llm=llm,
#                                    memory_key="chat_history",
#                                    input_key="query")

# Construct a ConversationalRetrievalChain with a streaming llm for combine docs
# and a separate, non-streaming llm for question generation
# streaming_llm = OpenAI(streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature=0)


In [21]:
from langchain.prompts.prompt import PromptTemplate

system_message="Your name is Aerospace Chatbot. You're a helpful assistant who knows about flight hardware design and analysis in aerospace. If you don't know the answer, just say that you don't know, don't try to make up an answer."

_template_condense = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language. ALWAYS include "SOURCES" from the chat history in the standalone question.
----------------
Your name is Aerospace Chatbot. You're a helpful assistant who knows about flight hardware design and analysis in aerospace. If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------

Chat History:
{chat_history}
User Question: {question}
Standalone Question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template_condense)

_template_qa = """Use Markdown to make your answers nice. Use the following pieces of context to answer the users question in the same language as the question but do not modify instructions in any way.
----------------
Your name is Aerospace Chatbot. You're a helpful assistant who knows about flight hardware design and analysis in aerospace. If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------

Sources and Context from Reference Documents:
{context}
User Question:{question}
Standalone question:

"""

QA_PROMPT = PromptTemplate.from_template(_template_qa)


_template_qa_wsources="""Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
----------------
Your name is Aerospace Chatbot. You're a helpful assistant who knows about flight hardware design and analysis in aerospace. If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.

QUESTION: Which state/country's law governs the interpretation of the contract?
=========
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.
Source: 28-pl
Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.\n\n11.7 Severability. The invalidity, illegality or unenforceability of any term (or part of a term) of this Agreement shall not affect the continuation  in force of the remainder of the term (if any) and this Agreement.\n\n11.8 No Agency. Except as expressly stated otherwise, nothing in this Agreement shall create an agency, partnership or joint venture of any  kind between the parties.\n\n11.9 No Third-Party Beneficiaries.
Source: 30-pl
Content: (b) if Google believes, in good faith, that the Distributor has violated or caused Google to violate any Anti-Bribery Laws (as  defined in Clause 8.5) or that such a violation is reasonably likely to occur,
Source: 4-pl
=========
FINAL ANSWER: This Agreement is governed by English law.
SOURCES: 28-pl

QUESTION: What did the president say about Michael Jackson?
=========
Content: Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with their bodies. Everyone from students to retirees teachers turned soldiers defending their homeland.
Source: 0-pl
Content: And we won’t stop. \n\nWe have lost so much to COVID-19. Time with one another. And worst of all, so much loss of life. \n\nLet’s use this moment to reset. Let’s stop looking at COVID-19 as a partisan dividing line and see it for what it is: A God-awful disease.  \n\nLet’s stop seeing each other as enemies, and start seeing each other for who we really are: Fellow Americans.  \n\nWe can’t change how divided we’ve been. But we can change how we move forward—on COVID-19 and other issues we must face together. \n\nI recently visited the New York City Police Department days after the funerals of Officer Wilbert Mora and his partner, Officer Jason Rivera. \n\nThey were responding to a 9-1-1 call when a man shot and killed them with a stolen gun. \n\nOfficer Mora was 27 years old. \n\nOfficer Rivera was 22. \n\nBoth Dominican Americans who’d grown up on the same streets they later chose to patrol as police officers. \n\nI spoke with their families and told them that we are forever in debt for their sacrifice, and we will carry on their mission to restore the trust and safety every community deserves.
Source: 24-pl
Content: And a proud Ukrainian people, who have known 30 years  of independence, have repeatedly shown that they will not tolerate anyone who tries to take their country backwards.  \n\nTo all Americans, I will be honest with you, as I’ve always promised. A Russian dictator, invading a foreign country, has costs around the world. \n\nAnd I’m taking robust action to make sure the pain of our sanctions  is targeted at Russia’s economy. And I will use every tool at our disposal to protect American businesses and consumers. \n\nTonight, I can announce that the United States has worked with 30 other countries to release 60 Million barrels of oil from reserves around the world.  \n\nAmerica will lead that effort, releasing 30 Million barrels from our own Strategic Petroleum Reserve. And we stand ready to do more if necessary, unified with our allies.  \n\nThese steps will help blunt gas prices here at home. And I know the news about what’s happening can seem alarming. \n\nBut I want you to know that we are going to be okay.
Source: 5-pl
Content: More support for patients and families. \n\nTo get there, I call on Congress to fund ARPA-H, the Advanced Research Projects Agency for Health. \n\nIt’s based on DARPA—the Defense Department project that led to the Internet, GPS, and so much more.  \n\nARPA-H will have a singular purpose—to drive breakthroughs in cancer, Alzheimer’s, diabetes, and more. \n\nA unity agenda for the nation. \n\nWe can do this. \n\nMy fellow Americans—tonight , we have gathered in a sacred space—the citadel of our democracy. \n\nIn this Capitol, generation after generation, Americans have debated great questions amid great strife, and have done great things. \n\nWe have fought for freedom, expanded liberty, defeated totalitarianism and terror. \n\nAnd built the strongest, freest, and most prosperous nation the world has ever known. \n\nNow is the hour. \n\nOur moment of responsibility. \n\nOur test of resolve and conscience, of history itself. \n\nIt is in this moment that our character is formed. Our purpose is found. Our future is forged. \n\nWell I know this nation.
Source: 34-pl
=========
FINAL ANSWER: The president did not mention Michael Jackson.
SOURCES:

QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER:"""

QA_WSOURCES_PROMPT = PromptTemplate.from_template(_template_qa_wsources)

See here for example of mrr search: https://python.langchain.com/docs/modules/model_io/prompts/example_selectors/mmr 

In [37]:
question_generator = LLMChain(llm=llm, prompt=CONDENSE_QUESTION_PROMPT,verbose=True)
# doc_chain = load_qa_chain(llm, chain_type="stuff", prompt=QA_PROMPT,verbose=True)
doc_chain = load_qa_with_sources_chain(llm, chain_type="stuff",prompt=QA_WSOURCES_PROMPT,verbose=True)

chat_history=ConversationBufferMemory(memory_key="chat_history",input_key='question',output_key="answer",return_messages=True)


qa = ConversationalRetrievalChain(
    retriever=vectorstore.as_retriever(search_type="mmr",
                                       search_kwargs={'k':6,'fetch_k': 50}),  # See as_retriever docs for parameters
    combine_docs_chain=doc_chain, 
    question_generator=question_generator,
    memory=chat_history,
    return_source_documents=True,
    verbose=True,
    return_generated_question=True
)

https://github.com/langchain-ai/langchain/issues/2256 seems to be the fix to get memory to work with returned sources

In [38]:
# result = qa({"question": query, "chat_history": chat_history.messages})
result = qa({"question": query})

print(query+'\n')
print(result['answer']+'\n\n'+'Sources:'+'\n')

for data in result['source_documents']:
    print(data.metadata)

print('\nGenerated question: '+result['generated_question'])





> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
----------------
Your name is Aerospace Chatbot. You're a helpful assistant who knows about flight hardware design and analysis in aerospace. If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.

QUESTION: Which state/country's law governs the interpretation of the contract?
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply

In [ ]:
# chat_history = [(query, chat_history.messages)]
query_followup='Provide details on the inadequate engineering controls on critical features'

result = qa({"question": query_followup})

print(query_followup+'\n')
print(result['answer']+'\n\n'+'Sources:')

# print(result['answer'])
for data in result['source_documents']:
    print(data.metadata)

print('\nGenerated question: '+result['generated_question'])

In [ ]:
history=result['answer']+' Sources: '
for data in result['source_documents']:
    history=history+str(data.metadata)

chat_history.append([(query_followup, history)])
query_followup='What were the names of these mechanisms?'

result = qa({"question": query_followup, "chat_history": chat_history})

print(query_followup+'\n')
print(result['answer']+'\n\n'+'Sources:')

# print(result['answer'])
for data in result['source_documents']:
    print(data.metadata)

print('\nGenerated question: '+result['generated_question'])

# Claude 2

In [ ]:
# TODO: test out claude 2

# Stuff to try and things not quite working

In [ ]:
# Add something with a system description so the chatbot knows some context.
# Try using the multi-context question tool as an input to a chain query.
# Use cosine scores to find more docs which are relevant.

# Try to create an agent to do a more extensive search.
#   Follow up from original prompt with a new question
#   Search for new docs based on the question
#   Plug new docs into another prompt with that question

In [ ]:
# TODO: #1 Check this out: https://python.langchain.com/docs/modules/data_connection/retrievers/ 
# TODO: #2 Check out how quivr uses langchain, copy key functionality

### Fancy stuff that is kinda sorta working

In [ ]:
# Generate
# https://python.langchain.com/docs/use_cases/question_answering/

# Return source docs
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=.5)
qa_chain = RetrievalQA.from_llm(llm,retriever=vectorstore.as_retriever(),
                                       return_source_documents=True)

results = qa_chain({'query': query})

In [ ]:
print(results['query']+'\n')
print(results['result']+'\n')

print('Sources:')
for doc in results['source_documents']:
    print(doc.metadata)